In [1]:
# Global Config Variables
n0 = 1000  # number of p=0 points in metric space
V = n0  # Threshold for p=0
#K = 10# No of clusters
A= 5# No of attributes 
J =2  #or 3 Number of demographic groups 
iterations = 70  # maximum iteration in clustering
runs =10
option='Kmeans'
dataset='Adult'


p_j = [13.384,6.6158]  #each can be max of p % in dataset   This will be \tau_i * k vector 
#[48.512,22.512,8.968]  0.2
# [53.536,26.463]    0.8  adult
#[25,12]   adult
#[13.384,6.6158] adult 0.2
#[25,10,4]  Bank 

In [2]:
import numpy as np
import os
import sys
import pandas
import math
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import log2
from sklearn import preprocessing
import sys
import timeit
import numba
from numba import jit , njit
from sklearn.metrics import pairwise_distances
from scipy.spatial import distance


In [ ]:
print(os.getcwd())
def load_Bank(data_dir=''):

    data_dir = data_dir
    _path = 'bank-full_p_6col.csv'
    data_path = os.path.join(data_dir, _path)

    K = 10

    df = pandas.read_csv(data_path, sep=',')
    #print(df.head())
    #print(len(df))
    
    return df
#load_Bank()

In [7]:
print(os.getcwd())
def load_Adult(data_dir=''):

    data_dir = data_dir
    _path = 'adult_p.csv'
    data_path = os.path.join(data_dir, _path)

    K = 10

    df = pandas.read_csv(data_path, sep=',')
    #print(df.head())
    #print(len(df))
    
    return df
#load_Adult()

/home/hpc128/Downloads/Fresh Experiments-Again/tau/Kmeans


In [8]:
if dataset=='Adult'
    df=load_Adult()
    df= df.round(decimals=5)
    print(len(df))
    df = df.dropna()
    print(len(df))
    typ = df['gender'].values
    c1 = np.count_nonzero(typ == 0)
    c2 = np.count_nonzero(typ == 1)
    n_j=[c1,c2]

    print(c1/(c1+c2))
    print(c2/(c1+c2))

    print(c1)
    print(c2)
elif dataset=='Bank':
    df=load_Bank()
    df= df.round(decimals=5)
    print(len(df))
    df = df.dropna()
    print(len(df))
    df['type'] = df['type']-1
    typ = df['type'].values
    #print(len(typ))
    #print(df.head(10))
    c1 = np.count_nonzero(typ == 0)
    c2 = np.count_nonzero(typ == 1)
    c3 = np.count_nonzero(typ == 2)
    n_j=[c1,c2,c3]

    print(c1/(c1+c2+c3))
    print(c2/(c1+c2+c3))
    print(c3/(c1+c2+c3))


    print(c1)
    print(c2)
    print(c3)


32561
32561
0.6692054912318418
0.33079450876815825
21790
10771


In [9]:
def dual_print(f,*args,**kwargs):
    #print(*args,**kwargs)
    print(*args,**kwargs,file=f)

def load_dataset(csv_name):
    # read the dataset from csv_name and return as pandas dataframe
    df = pd.read_csv(csv_name, header=None)
    return df


def k_random_index(df,K):
    # return k random indexes in range of dataframe
    return random.sample(range(0, len(df)), K)

def find_k_initial_centroid(df,K):
    centroids = []    # make of form [ [x1,y1]....]

    rnd_idx = k_random_index(df,K)
    #print(rnd_idx)
    for i in rnd_idx:
        coordinates =[]
        for a in range(0,A):
            coordinates.append(df.loc[i][a])
        centroids.append(coordinates)   #df is X,Y,....., Type

    return centroids

def find_k_initial_centroid_old(df):
    centroids = []    # make of form [ [x1,y1]....]

    rnd_idx = k_random_index(df)
    #print(rnd_idx)
    for i in rnd_idx:
        coordinates =[]
        for a in range(0,A):
            coordinates.append(df.loc[i][a])
        centroids.append(coordinates)   #df is X,Y,....., Type

    return centroids

#nOt using
def calc_distance(x1, y1, x2, y2):
    # returns the euclidean distance between two points
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

def calc_distance_a(centroid, point):
    #print('çalculating distance\n')

    sum_ = 0

    for i in range(0, len(centroid)):
        sum_ = sum_ + (centroid[i]-point[i])**2

    return sum_ #**0.5
@njit(parallel=False)
def find_distances_fast(k_centroids, df):
    
    dist = np.zeros((len(k_centroids),len(df),A+2),np.float64)
    Kcnt = 0 
    for c in k_centroids:  #K-centroid is of form [ c1=[x1,y1.....z1], c2=[x2,y2....z2].....]
       
        l = np.zeros((len(df),A+2),np.float64)
        
      
        index = 0 
        for row in df:                # row is now x,y,z......type
            # append all coordinates to point
            dis = np.sum((c- row[:A])**2)#calc_distance_a(c, point)
            #Processing the vector for list
            row_list = np.array([dis])
            #append distance or l norm
            row_list = np.append(row_list,row[:A+1])
            #append all coordinates #append type of this row
  
            l[index] = row_list
            index = index + 1
            #l.append([calc_distance(c[0], c[1], row[0], row[1]), row[0], row[1], row[2]])  # [dist, X, Y,....Z , type]
            # l contains list of type [dist,X,Y.....,Z,type] for each points in metric space
        dist[Kcnt]= l
        Kcnt = Kcnt + 1

    # return dist which contains distances of all points from every centroid

    return dist


def find_distances(k_centroids, df):
    dist = []
    for c in k_centroids:  #K-centroid is of form [ c1=[x1,y1.....z1], c2=[x2,y2....z2].....]
        l = []
     

        for index, row in df.iterrows():                # row is now x,y,z......type
            point =[]
            for a in range(0, A):
                point.append(row.iloc[a])  # append all coordinates

            dis = calc_distance_a(c, point)
            #Processing the vector for list
            row_list = [dis]
            #append distance or l norm

            for a in range(0, A):
                row_list.append(row.iloc[a])   #append all coordinates
           
            row_list.append(row.iloc[a+1])   #append type of this row

            l.append(row_list)
            #l.append([calc_distance(c[0], c[1], row[0], row[1]), row[0], row[1], row[2]])  # [dist, X, Y,....Z , type]
            # l contains list of type [dist,X,Y.....,Z,type] for each points in metric space
        dist.append(l)

    
    
    return dist


def sort_and_valuation(dist):
    sorted_val = []
    for each_centroid_list in dist:
        each_centroid_list_sorted = sorted(each_centroid_list, key=lambda x: (x[A+1], x[0]))  # A+1 is index of type , 0 is dist
        sorted_val.append(each_centroid_list_sorted)

        # sort on basis of type & then dist.
        # Now all whites are towards start and all black are after white as they have additional V added to their valuation
        # Among the whites, the most closest is at start of list as it has more valuation.
        # Similarly sort the black points among them based on distance as did with white

    return sorted_val


def clustering(sorted_valuation, hashmap_points,k_centroid,n,K):
    n = n#len(hashmap_points.keys())  # total number of points in metric space
    cluster_assign = []

    for i in range(0, K):
        cluster_assign.append([])  # initially all clusters are empty

    tot_p_j =0 
    curr_cluster =0
    curr_j=0

    curr_alloc_j= []
    for j in range(0,J):
        curr_alloc_j.append(0);

    curr_alloc_j = np.array(curr_alloc_j)

    tot_point_alloc_all_j = np.sum(np.floor(np.array(p_j)*K*n_j/100))
    
    print("Total points to allocate for current j are  :"+str(tot_point_alloc_all_j))
    
    
    while( tot_point_alloc_all_j !=  np.sum(curr_alloc_j)):
        
        j_tot_need = np.floor(K*p_j[curr_j]*n_j[j]/100)  #check how many points for j we need to assing in all to cluster

        
        if curr_alloc_j[curr_j] == j_tot_need:   #curr alloc maintain for all k count of j 
            curr_j =(curr_j+1) % J
            print("Working on J = J+1")
        
                   
        index=0
        each = sorted_valuation[curr_cluster % K][index]

        while each[-1] != curr_j:
            index = index +1
            each = sorted_valuation[curr_cluster % K][index]

        

        while(index != len( sorted_valuation[curr_cluster % K] )):
            
            each = sorted_valuation[curr_cluster % K][index]
            
          
            if tuple(each[1:]) in hashmap_points.keys():# each is (dist, X,Y,....Z, type)
          
                if int(hashmap_points[tuple(each[1: ])]) > 0:    #if some points of that type left to allocate
                    cluster_assign[curr_cluster].append(each)
                    hashmap_points[tuple(each[1: ])] -= 1
                    curr_alloc_j[curr_j]+=1
                    break
                    
            index = index +1 

        curr_cluster = (curr_cluster+1)%K   
        
            
        
    print("Finding the left out points after allocation for p%")
  #left out in hashmap points ko put in something and allocate them. 
    left_out_points = []

    
    
    for each in sorted_valuation[0]:           #check all points using any sorted val that whcih points are not allocated to any cluster using map dict. using sorted jsut need key for dict
        if hashmap_points[tuple(each[1:])] > 0:  #if still unallocated point 
            left_out_points.append(each)                    #each is (dist, X,Y,....Z, type)
        
    
    
    
   
    
    print("Total left out points are "+str(n-tot_point_alloc_all_j))
    if len(left_out_points)>0:
        left_out_points_= np.delete(left_out_points,0,1)  # colno , axis
        left_out_points__= np.delete(left_out_points_,-1,1)  # colno , axis
        distances_to_centroids_left_points = distance.cdist(left_out_points__, k_centroid, 'sqeuclidean')
        cluster_assignment_left_points = np.argmin(distances_to_centroids_left_points, axis = 1)
        ch =0
        for ind in range(0,len(left_out_points)):
            each = left_out_points[ind]
            each_ = left_out_points_[ind]
            each__ = left_out_points__[ind]
            if int(hashmap_points[tuple(each[1:])]) > 0: 
                hashmap_points[tuple(each[1: ])] -= 1
                cluster_assign[cluster_assignment_left_points[ind]].append(each)
                ch = ch +1
        print("Total points alloced  are:"+str(ch+tot_point_alloc_all_j)) 

   
    check=0
    for value in hashmap_points.values():
        if  value >0:
            check +=1
    
    print("Total points left  are:"+str(check))
    print("Total points in dataset are:"+str(n))
    
    return cluster_assign


def update_centroids(cluster_assign,K):

    new_centroids = []
    for k in range(0, K):

        sum_a = []

        for i in range(0, A):
            sum_a.append(0)

        for each in cluster_assign[k]:
            sum_a = [sum(x) for x in zip(sum_a, each[1:-1])]
            #each is (dist,X,Y,.....Z,type)
       # print('sum a is '+str(sum_a))
        new_coordinates = []
        for a in range(0, A):
            new_coordinates.append(sum_a[a] / len(cluster_assign[k]))
        new_centroids.append(new_coordinates)
        k=k+1



    return new_centroids


def update_centroids_median(cluster_assign,K):
    new_centroids = []
    for k in range(0, K):

        cAk =  np.array(cluster_assign[k])
        cAk = np.delete(cAk,[0,-1],axis=1)
      
        if len(cAk) %2 ==0 and len(cAk)>0: 
            cc = [np.median(np.array(cAk[:-1])[:,cl]) for cl in range(0,cAk.shape[1])]
            new_centroids.append(cc)
        elif len(cAk) %2 !=0 and len(cAk)>0:
            cc = [np.median(np.array(cAk)[:,cl]) for cl in range(0,cAk.shape[1])]
            new_centroids.append(cc)
        elif len(cAk)==0:
            print("Error: No centroid found updation error")
    
    return new_centroids
        

def calc_clustering_objective(k_centroid, cluster_assign,K):
    cost = 0

    for k in range(0, K):

        for each in cluster_assign[k]:  #each is (dist, X,Y,....,Z,type)
            dd = calc_distance_a(k_centroid[k], each[1:-1])
            cost = cost + (dd)
#            print('K: '+ str(k)+ 'Each '+ str(each[1:-1])+ ' centroid ; '+ str(k_centroid)+ 'dist : '+ str(dd))

    return cost




In [12]:

list_fair_run=[]
list_obj_run =[]       
list_balance_run=[]

def main():
    # Step1 : Load the dataset
  
    list_fair_K=[]
    list_obj_K =[]       
    list_balance_K=[]
    
    if(not os.path.isdir('Adult_Runs_p_per')):
        os.makedirs('Adult_Runs_p_per')
    for kk in [10,2,5,15,20,30,40]:
        K = kk
        
        print(" K=="+str(K)+"  ")
        
        list_fair_run=[]
        list_obj_run =[]       
        list_balance_run=[]
        list_time_run=[]
        list_iterNo_run =[]
        seeds = [0,100,200,300,400,500,600,700,800,900,1000,1100]
      
        for run in range(0,runs):
            np.random.seed(seeds[run])
            random.seed(seeds[run])
            f = open('Adult_Runs_p_per/K_'+str(K)+'_run_'+str(run)+'_output.txt', 'a')
            print("+"*100)
            print('                        RUN  : '+ str(run))


            list_fair_iter=[]
            list_obj_iter =[]
            list_balance_iter=[]

   
            # Step2 : Find initial K random centroids using k_random_index(df) & find_k_initial_centroid(df)
            k_centroid = find_k_initial_centroid(df,K)
       
            prev_assignment =[]
            cluster_assignment = []

            for i in range(0, K):
                cluster_assignment.append([])  # initially all clusters are empty

            sum_time = 0
            curr_itr = 0
            prev_objective_cost=-1
            objective_cost = 0
            # Step3 : Find distances from the centroids using find_distances() with list of [ [x1,y1,z1..] , [x2,y2,z2..]....] centroids format list
            while curr_itr <= iterations:# and prev_objective_cost != objective_cost:

                start = timeit.default_timer()

              
                dual_print(f,'Calulating distance for iteration : '+ str(curr_itr)+'\n')
                df1 = df.values
                k_centroids1= np.array(k_centroid)
                dist = find_distances_fast(k_centroids1, df1)
                
                dual_print(f,'Finished calc distance for iteration : '+ str(curr_itr)+'\n')
                # Step4 :  Find Valuation matrix for all centroids using sort_and_valuation()
                dual_print(f,'Calulating Valuation for iteration : '+ str(curr_itr)+'\n')

                valuation = sort_and_valuation(dist)
                dual_print(f,'Finished Valuation for iteration : '+ str(curr_itr)+'\n')

                #Step5 : Perform clustering using valuation matrix & hashmap of all points in metric
                hash_map = {}
            
                for index, row in df.iterrows():
                    temp = tuple(row[:-1])
                    
                    if tuple(row) in hash_map.keys():
                        hash_map[tuple(row)] = int(hash_map[tuple(row)]) + 1
                    else:
                         hash_map.update({tuple(row): int(1)})   #dict is of form { (x,y): 0 , ....}
            
                
                
                
                dual_print(f,'Finding clusters for iteration : '+ str(curr_itr)+'\n')

                prev_assignment = cluster_assignment
                cluster_assignment = clustering(valuation, hash_map,k_centroid,len(dist[0]),K)

                dual_print(f,'Finished finding cluster for iteration : '+ str(curr_itr)+'\n')

              
                clustering_cost = calc_clustering_objective(k_centroid,cluster_assignment,K)
                if curr_itr!=0:
                     prev_objective_cost = objective_cost
                    
                objective_cost = np.round(clustering_cost,3)


              
               
                list_obj_iter.append(str(objective_cost))
              

                dual_print(f,'Clustering Objective/Cost ' + str(clustering_cost) + '\n')
                
                if abs(objective_cost-prev_objective_cost) <= 200:
                    break;
                #Step6 : Print the cluster assignments

                dual_print(f,'Updating centroids for iteration : '+ str(curr_itr)+'\n')

                #Step7 : Find new centroids using mean of all points in current assignment
                if option=='Kmeans':
                    
                    k_centroid = update_centroids(cluster_assignment,K)
                else:
                    k_centroid = update_centroids_median(cluster_assignment,K)
                dual_print(f,'Finished centroid updation for iteration : '+ str(curr_itr)+'\n')
                dual_print(f,'Iteration No: '+str(curr_itr)+' : updated centroid are : '+ str(k_centroid))
                #Step8 : Repeat from Step3 until clusters are same or iterations reach upper limit
                stop = timeit.default_timer()
                sum_time += (stop - start)
              
                dual_print(f,'Time for iteration : ' + str(curr_itr) + ' is  ' + str(stop - start) + '\n')

                curr_itr += 1

                dual_print(f,'-----------------------------Finished-----------------------------------------------\n')


                print('-----------------------------Finished-----------------------------------------------\n')

            
            list_time_run.append(sum_time)
            list_iterNo_run.append(curr_itr-1)
            dual_print(f,'Total time taken is '+ str(sum_time)+'\n')
            dual_print(f,'iertations total : '+str(curr_itr-1))
            
            clustering_cost_converged = calc_clustering_objective(k_centroid,cluster_assignment,K)

            print("\nCost variation over iterations")
            print(list_obj_iter)
            

            f.close()
            run  = run +1
            list_obj_run.append(clustering_cost_converged)
            
            
        
        print("@"*70)
        print("Cost variations over run")
        print(str(list_obj_run))
        
        print("Mean Cost variations over run")
        print(str(np.mean(np.array(list_obj_run))))
        print("Std Dev Cost variations over run")
        print(str(np.std(np.array(list_obj_run))))
        print("#"*30)
        print("Mean Time variation over run")
        print(str(np.mean(np.array(list_time_run))))
        print("Std Dev time variation over run")
        print(str(np.std(np.array(list_time_run))))
        print("#"*30)
        print("Mean iterations taken over run")
        print(str(np.mean(np.array(list_iterNo_run))))
        print("Std Dev iteraions taken  over run")
        print(str(np.std(np.array(list_iterNo_run))))
        print("#"*30)
        
        list_obj_K.append(np.mean(np.array(list_obj_run)))
       
        
    print("%"*70)
    print("Cost variations over K")
    print(str(list_obj_K))
   
    print("#"*30)


In [13]:


if __name__ == "__main__":
    main()
   


 K==10  
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                        RUN  : 0
Total points to allocate for current j are  :6511.0
Working on J = J+1
Finding the left out points after allocation for p%
Total left out points are 26050.0
Total points alloced  are:32561.0
Total points left  are:0
Total points in dataset are:32561
-----------------------------Finished-----------------------------------------------

Total points to allocate for current j are  :6511.0
Working on J = J+1
Finding the left out points after allocation for p%
Total left out points are 26050.0
Total points alloced  are:32561.0
Total points left  are:0
Total points in dataset are:32561
-----------------------------Finished-----------------------------------------------

Total points to allocate for current j are  :6511.0
Working on J = J+1
Finding the left out points after allocation for p%
Total left out points are 26050.0
Total points alloced  are:32

-----------------------------Finished-----------------------------------------------

Total points to allocate for current j are  :6511.0
Working on J = J+1
Finding the left out points after allocation for p%
Total left out points are 26050.0
Total points alloced  are:32561.0
Total points left  are:0
Total points in dataset are:32561
-----------------------------Finished-----------------------------------------------

Total points to allocate for current j are  :6511.0
Working on J = J+1
Finding the left out points after allocation for p%
Total left out points are 26050.0
Total points alloced  are:32561.0
Total points left  are:0
Total points in dataset are:32561
-----------------------------Finished-----------------------------------------------

Total points to allocate for current j are  :6511.0
Working on J = J+1
Finding the left out points after allocation for p%
Total left out points are 26050.0
Total points alloced  are:32561.0
Total points left  are:0
Total points in dataset ar

-----------------------------Finished-----------------------------------------------

Total points to allocate for current j are  :6511.0
Working on J = J+1
Finding the left out points after allocation for p%
Total left out points are 26050.0
Total points alloced  are:32561.0
Total points left  are:0
Total points in dataset are:32561
-----------------------------Finished-----------------------------------------------

Total points to allocate for current j are  :6511.0
Working on J = J+1
Finding the left out points after allocation for p%
Total left out points are 26050.0
Total points alloced  are:32561.0
Total points left  are:0
Total points in dataset are:32561
-----------------------------Finished-----------------------------------------------

Total points to allocate for current j are  :6511.0
Working on J = J+1
Finding the left out points after allocation for p%
Total left out points are 26050.0
Total points alloced  are:32561.0
Total points left  are:0
Total points in dataset ar